In [85]:
%matplotlib notebook
import pandas as pd
import mne
import numpy as np
import seaborn as sns
import helpers
import subject_DM
import statsmodels.api as sm
from scipy.stats import ttest_1samp

# Compute baseline activity for all subjects

In [4]:
chtype = 'mag'
bl = (-0.3, 0.0)

subjects = helpers.find_available_subjects(megdatadir=helpers.megdatadir)
S = len(subjects)

In [60]:
baseline_all = []
for sub in subjects:
    print('loading subject %2d' % sub)
    # load MNE epochs
    epochs = helpers.load_meg_epochs_from_sdat(sub, helpers.megdatadir)

    # pick specific channels
    epochs = epochs.pick_types(meg='mag')

    df = epochs.to_data_frame()
    df.index = pd.MultiIndex.from_product(
        [np.arange(480)+1, df.index.levels[2]], names=['trial', 'time'])
    
    baseline_all.append(df.loc[(slice(None), slice(int(bl[0]*1000), int(bl[1]*1000))), :].mean(level='trial'))
    
baseline_all = pd.concat(baseline_all, keys=subjects, names=['subject'])
baseline_all_norm = (baseline_all - baseline_all.mean()) / baseline_all.std()

loading subject  2
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject  3
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject  4
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject  5
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject  6
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject  7
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject  8
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject  9
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject 10
480 matching events found
0 bad epochs dropped
Converting time column to int64...
loading subject 11
480 matching events found
0 bad epochs dropped
Converting time column to

# Correlate baseline activity with regressors

## define regressors

In [75]:
r_names = ['abs_dot_y', 'abs_dot_x', 'dot_y', 'dot_x', 'entropy', 'trial_time', 
           'intercept', 'response', 'dot_x_cflip']

dots = np.arange(1, 6)

rt_thresh = 0.1

if 'RT' in r_names:
    DM = subject_DM.get_trial_DM(dots, r_names=r_names)
else:
    DM = subject_DM.get_trial_DM(dots, r_names=r_names+['RT'])
    
DM = DM.loc(axis=0)[subjects, :]

good_trials = DM['RT'] >= helpers.dotdt*(dots.max()-1) + rt_thresh

if 'RT' not in r_names:
    del DM['RT']
    
DM = DM.loc[good_trials]

DM.sort_index(axis=1, inplace=True)
R = DM.shape[1]

normalise

In [77]:
DM = (DM - DM.mean()) / DM.std()
# intercept will be nan, because it has no variance
DM['intercept'] = 1

## run GLM

In [86]:
first_level = pd.DataFrame([], 
        index=baseline_all.columns,
        columns=pd.MultiIndex.from_product([subjects, ['beta', 'bse'], 
              DM.columns], names=['subject', 'measure', 'regressor']), 
              dtype=np.float64)
first_level.sort_index(axis=1, inplace=True)

second_level = pd.DataFrame([], 
        index=baseline_all.columns, 
        columns=pd.MultiIndex.from_product([['mean', 'std', 'mlog10p'], 
              DM.columns], names=['measure', 'regressor']), dtype=np.float64)
second_level.sort_index(axis=1, inplace=True)

for channel in baseline_all_norm.columns:
    print('computing channel ' + channel + ' ...')
    
    data = baseline_all_norm.loc[:, channel]
    data = data.loc[good_trials.values]

    params = np.zeros((S, R))
    for s, sub in enumerate(subjects):
        res = sm.OLS(data.loc[sub].values, DM.loc[sub].values, 
                     hasconst=True).fit()
        params[s, :] = res.params

        first_level.loc[channel, (sub, 'beta', slice(None))] = (
            params[s, :])
        first_level.loc[channel, (sub, 'bse', slice(None))] = (
            res.bse)

    second_level.loc[channel, ('mean', slice(None))] = (
            params.mean(axis=0))
    second_level.loc[channel, ('std', slice(None))] = (
            params.std(axis=0))
    _, pvals = ttest_1samp(params, 0, axis=0)
    second_level.loc[channel, ('mlog10p', slice(None))] = (
            -np.log10(pvals))

# Results

In [101]:
measure = 'mean'

second_level.xs(measure, axis=1, level='measure').abs().max().sort_values()

regressor
abs_dot_y_1      0.011849
abs_dot_x_4      0.019495
dot_y_3          0.019905
intercept        0.020986
dot_y_5          0.021576
dot_x_cflip_5    0.021702
abs_dot_y_3      0.021705
abs_dot_y_4      0.023432
dot_y_2          0.024063
dot_y_4          0.024425
abs_dot_x_5      0.024712
dot_x_cflip_1    0.024744
dot_x_cflip_2    0.026133
dot_x_cflip_3    0.026900
abs_dot_x_2      0.027672
abs_dot_y_5      0.028879
abs_dot_x_3      0.029128
abs_dot_y_2      0.029682
trial_time       0.030247
dot_x_cflip_4    0.032756
abs_dot_x_1      0.038001
dot_y_1          0.043259
response         0.044406
entropy          0.054246
dot_x_5          0.069555
dot_x_2          0.071440
dot_x_3          0.074581
dot_x_4          0.077874
dot_x_1          0.103972
dtype: float64